# **PROJET FINE-TUNING LLM**

## ***Importer les librairies usuelles*** ##

In [1]:
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install trl
!pip install rouge-score


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-6qtur29r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-6qtur29r
  Resolved https://github.com/huggingface/transformers.git to commit 5fa35344755d8d9c29610b57d175efd03776ae9e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-gjqgwuq_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-gjqgwuq_
  Resolved https://github.com/huggingface/peft.git to commit aa3f41f7529ed078e9225b2fc1edbb8c71f58f99
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-wmso1qd2
  Runni

In [2]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset
import random

from transformers import GPT2LMHeadModel, GPTNeoForCausalLM, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge_score import rouge_scorer


/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


## ***Importer le datset*** ##

In [3]:
dataset = load_dataset("manu/french_poetry")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Extraire le Datset grâce à la clé "train"
df = dataset['train'].to_pandas()
df.head()

,title,poet,text,link,id
0,Les deux bonnes soeurs,Charles Baudelaire (1821-1867),Poésie : Les deux bonnes soeurs\nTitre : Les d...,https://www.poesie-francaise.fr/charles-baudel...,charles-baudelaire-poeme-les-deux-bonnes-soeurs
1,"Ami, j'ai quitté vos fêtes",Victor Hugo (1802-1885),"Poésie : Ami, j'ai quitté vos fêtes\nTitre : A...",https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-ami-j-ai-quitte-vos-fetes
2,Je ne vois pas pourquoi je ferais autre chose,Victor Hugo (1802-1885),Poésie : Je ne vois pas pourquoi je ferais aut...,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-je-ne-vois-pas-pourquoi-je-f...
3,À Mlle Fanny de P,Victor Hugo (1802-1885),Poésie : À Mlle Fanny de P\nTitre : À Mlle Fan...,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-a-Mlle-Fanny-de-P
4,L'angélus du matin,Paul Verlaine (1844-1896),Poésie : L'angélus du matin\nTitre : L'angélus...,https://www.poesie-francaise.fr/paul-verlaine/...,paul-verlaine-poeme-langelus-du-matin


## ***Explorer le Dataset***

In [5]:
df.dtypes

,0
title,object
poet,object
text,object
link,object
id,object


In [6]:
df.shape

(1821, 5)

In [7]:
df['text'].iloc[50]

"Poésie : Un dahlia\nTitre : Un dahlia\nPoète : Paul Verlaine (1844-1896)\nRecueil : Poèmes saturniens (1866).\nCourtisane au sein dur, à l'oeil opaque et brun\nS'ouvrant avec lenteur comme celui d'un boeuf,\nTon grand torse reluit ainsi qu'un marbre neuf.\nFleur grasse et riche, autour de toi ne flotte aucun\nArôme, et la beauté sereine de ton corps\nDéroule, mate, ses impeccables accords.\nTu ne sens même pas la chair, ce goût qu'au moins\nExhalent celles-là qui vont fanant les foins,\nEt tu trônes, Idole insensible à l'encens.\n— Ainsi le Dahlia, roi vêtu de splendeur,\nElève sans orgueil sa tête sans odeur,\nIrritant au milieu des jasmins agaçants !"

## ***Effectuer le preprocessing sur le dataset*** ##

In [8]:
# Récupérer la section poème positionnée à l'indice 4
df['text'] = df['text'].str.split(':').str[4]

# Garder la section poème positionnée après le premier "\n"
df['text'] = df['text'].str.split('\n', n=1).str[1]

# Supprimer l'intégralité des NAN (poèmes structure non conforme)
df['text'] = df['text'].dropna()

In [9]:
df.head(15)

,title,poet,text,link,id
0,Les deux bonnes soeurs,Charles Baudelaire (1821-1867),Sonnet.\nLa Débauche et la Mort sont deux aima...,https://www.poesie-francaise.fr/charles-baudel...,charles-baudelaire-poeme-les-deux-bonnes-soeurs
1,"Ami, j'ai quitté vos fêtes",Victor Hugo (1802-1885),NaN,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-ami-j-ai-quitte-vos-fetes
2,Je ne vois pas pourquoi je ferais autre chose,Victor Hugo (1802-1885),Je ne vois pas pourquoi je ferais autre chose\...,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-je-ne-vois-pas-pourquoi-je-f...
3,À Mlle Fanny de P,Victor Hugo (1802-1885),NaN,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-a-Mlle-Fanny-de-P
4,L'angélus du matin,Paul Verlaine (1844-1896),NaN,https://www.poesie-francaise.fr/paul-verlaine/...,paul-verlaine-poeme-langelus-du-matin
5,J'ai tout donné pour rien,Théophile Gautier (1811-1872),"On peut m'aimer sans honte,\nLa couronne de co...",https://www.poesie-francaise.fr/theophile-gaut...,theophile-gautier-poeme-jai-tout-donne-pour-rien
6,La dure épreuve va finir,Paul Verlaine (1844-1896),La dure épreuve va finir,https://www.poesie-francaise.fr/paul-verlaine/...,paul-verlaine-poeme-la-dure-epreuve-va-finir
7,À deux beaux yeux,Théophile Gautier (1811-1872),Vous avez un regard singulier et charmant ;\nC...,https://www.poesie-francaise.fr/theophile-gaut...,theophile-gautier-poeme-a-deux-beaux-yeux
8,Écrit en 1827,Victor Hugo (1802-1885),NaN,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-ecrit-en-1827
9,Lorsque l'enfant paraît,Victor Hugo (1802-1885),NaN,https://www.poesie-francaise.fr/victor-hugo/po...,victor-hugo-poeme-lorsque-l-enfant-parait


In [10]:
df.shape

(1821, 5)

## ***Suppression des valeurs NaN aprés transformation ne respectant pas la structure globale***

In [11]:
# Détecter les valeurs NaN
missing_values = df.text.isnull().sum()
print(missing_values)

412


In [12]:
# Supprimer les valeurs Nan
df = df.dropna(subset=['text']).reset_index(drop =True)

In [13]:
df.shape

(1409, 5)

In [14]:
# Compter les valeurs NaN après suppression
missing_values = df.text.isnull().sum()
print(missing_values)

0


In [15]:
df.shape

(1409, 5)

In [16]:
df['text'].iloc[100]

"Et nos longs entretiens qui font les soirs si courts,\nTon vénérable amour que jamais rien n'émousse\nPour toute belle chose et toute chose douce !\nMaint poème charmant que nous disait ta voix\nM'apparaît... — Mon esprit, admirant à la fois\nTant de jours sur ton front, tant de grâce en ton style,\nCroit voir un patriarche au milieu d'une idylle !\nAinsi tu n'es jamais loin de mon âme, et puis\nTout me parle de toi dans ces champs où je suis ;\nJe compare, en mon coeur que ton ombre accompagne,\nTa verte poésie et la fraîche campagne ;\nJe t'évoque partout ; il me semble souvent\nQue je vais te trouver dans quelque coin rêvant,\nEt que, dans le bois sombre ouvrant ses ailes blanches,\nTon vers jeune et vivant chante au milieu des branches.\nJe m'attends à te voir sous un arbre endormi.\nJe dis "

In [17]:
# Suppression des colonnes inutiles
df = df.drop(columns = ['poet', 'link', 'id'])

In [18]:
# Dictionnaire des prompts utilisés pour améliorer la robustesse du dataset
prompt_variance = [
    "Écris un poème sur le thème : {title}",
    "Compose un poème intitulé : {title}",
    "Imagine un poème ayant pour sujet : {title}",
    "Crée un poème autour de : {title}",
    "Un poème inspiré de {title}, s'il te plaît",
    "{title}",
    "Rédige un poème poétique à propos de : {title}",
    "Propose un poème sur le sujet suivant : {title}",
    "Donne-moi un poème avec pour thème principal : {title}",
    "Développe un poème basé sur : {title}",
    "Invente un poème portant sur : {title}",
    "Un poème intitulé {title}",
    "Sur le thème {title}, écris un poème captivant",
    "Conçois un poème ayant comme sujet : {title}",
    "Pour {title}, écris un poème unique",
    "À partir du sujet '{title}', rédige un poème",
    "Avec {title}, inspire-toi pour créer un poème",
    "Un poème autour de {title}, si possible",
    "Donne-moi un poème avec pour thème {title}",
    "Fais un poème en explorant le thème {title}",
    "Compose une œuvre poétique sur {title}",
    "Rédige un poème à propos de {title}"
]

In [19]:
# Fixer une seed pour la reproductibilité des résultats
random.seed(42)

# Applqiuer les différents prompt aléatoirement sur les titres des poèmes
df['prompt'] = df['title'].apply(lambda title: random.choice(prompt_variance).format(title=title))

In [20]:
# Conserver uniquement les colonnes prompt et text
df = df[['prompt', 'text']]

In [21]:
df.head()

,prompt,text
0,Compose une œuvre poétique sur Les deux bonnes...,Sonnet.\nLa Débauche et la Mort sont deux aima...
1,Crée un poème autour de : Je ne vois pas pourq...,Je ne vois pas pourquoi je ferais autre chose\...
2,Écris un poème sur le thème : J'ai tout donné ...,"On peut m'aimer sans honte,\nLa couronne de co..."
3,Donne-moi un poème avec pour thème principal :...,La dure épreuve va finir
4,Propose un poème sur le sujet suivant : À deux...,Vous avez un regard singulier et charmant ;\nC...


In [22]:
# Créer une colonne avec la longueur de chaque séquence
df['text_length'] = df['text'].apply(len)

In [23]:
import plotly.express as px

# Etudier la répartition des length pour trouver la taille de séquence optimale
fig = px.histogram(
    df,
    x="text_length",
    title="Histogramme de répartition des longuers de séquence",
    labels={"values": "Valeurs"},
    nbins=30
)
fig.show()


In [24]:
# Créer un mask pour garder les séquences inferieur à 1500 caractères
mask = df['text_length'] < 1500
row_lenght_threshold = df[mask]

In [25]:
row_lenght_threshold.shape[0]

1330

In [26]:
# Appliquer le mask sur le dataset
df = df[mask]

In [27]:
# Longueur de séquence maximale
df.text_length.max()

1489

In [28]:
df.shape

(1330, 3)

In [29]:
df.head()

,prompt,text,text_length
0,Compose une œuvre poétique sur Les deux bonnes...,Sonnet.\nLa Débauche et la Mort sont deux aima...,667
1,Crée un poème autour de : Je ne vois pas pourq...,Je ne vois pas pourquoi je ferais autre chose\...,448
2,Écris un poème sur le thème : J'ai tout donné ...,"On peut m'aimer sans honte,\nLa couronne de co...",141
3,Donne-moi un poème avec pour thème principal :...,La dure épreuve va finir,25
4,Propose un poème sur le sujet suivant : À deux...,Vous avez un regard singulier et charmant ;\nC...,654


Tokens spéciaux : {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}

## ***Ajouter les token spéciaux au dataset qui ne sont pas présent par déafut dans le tokenizer*** ##

In [30]:
# token saut de ligne
df['text'] = df["text"].str.replace("\n", "<|newline|>", regex = False)

# token padding
# Ajouter des tokens <|pad|> pour chaque ligne
target_length = 1489

df["padded_text"] = df["text"] + df.apply(
    lambda row: " " + " ".join(["<|pad|>"] * (target_length - row["text_length"]))
    if row["text_length"] < target_length else "", axis=1
)

<ipython-input-30-12bacc327462>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-30-12bacc327462>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
# Conserver uniquement les colonnes prompt et text
df = df[['prompt', 'text']]

In [32]:
df.head()

,prompt,text
0,Compose une œuvre poétique sur Les deux bonnes...,Sonnet.<|newline|>La Débauche et la Mort sont ...
1,Crée un poème autour de : Je ne vois pas pourq...,Je ne vois pas pourquoi je ferais autre chose<...
2,Écris un poème sur le thème : J'ai tout donné ...,"On peut m'aimer sans honte,<|newline|>La couro..."
3,Donne-moi un poème avec pour thème principal :...,La dure épreuve va finir
4,Propose un poème sur le sujet suivant : À deux...,Vous avez un regard singulier et charmant ;<|n...


In [33]:
df.text[5]

"Ah ! Si vous saviez comme on pleure<|newline|>De vivre seul et sans foyers,<|newline|>Quelquefois devant ma demeure<|newline|>Vous passeriez.<|newline|>Si vous saviez ce que fait naître<|newline|>Dans l'âme triste un pur regard,<|newline|>Vous regarderiez ma fenêtre<|newline|>Comme au hasard.<|newline|>Si vous saviez quel baume apporte<|newline|>Au cœur la présence d'un cœur,<|newline|>Vous vous assoiriez sous ma porte<|newline|>Comme une sœur.<|newline|>Si vous saviez que je vous aime,<|newline|>Surtout si vous saviez comment,<|newline|>Vous entreriez peut-être même<|newline|>Tout simplement."

In [34]:
# Convertir le dataset en CSV
df.to_csv('poems_prompts_dataset.csv', index=False, encoding='utf-8')

## ***Importer le modèle pour le tester sans fine-tuning*** ##

In [35]:
#df = pd.read_csv('poems_prompts_dataset.csv')

In [36]:
from transformers import pipeline
generator = pipeline('text-generation', model='Gragroo/gpt2-french-small-finetuned-2024-09-21')
generator("EleutherAI has", do_sample=True, min_length=50)

config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/858k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/517k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

Device set to use cpu


[{'generated_text': "EleutherAI has a lieu le 15 Mai 2024 le syndicat en centre de référence du service public de transports de l’Ain (CMP) de Lyon. À la suite de l'appel d'appel du 10 Septembre"}]

location model on machine : C:\Users\33760\.cache\huggingface\transformers\

In [37]:
prompt = "Génère un poème sur l'amitié"
res = generator(prompt, max_length = 200, top_k = 80, top_p = 0.90, do_sample = True, temperature = 1.1)

print(res[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Génère un poème sur l'amitié et la haine », prononcé par la décision enregistrée par laquelle l'établissement public dispose d’un pouvoir spécial



exécutoires
délivrées le:


■




PCP JCP ACR fond
 
N° RG 24/02897 - 
N° Portalis 352J-W-B7I-CX3C4U

N° MINUTE : 

Assignation du :
21 Juillet 2024






JUGEMENT 
rendu le 18 Mai 2024 
DEMANDERESSE
S.C.I. GUGEBILION
[Adresse 3]
[Localité 4]

représentée par Me Catherine HEN-LANIGARD du REPOYLET MAPRIS GUGE


In [38]:
generator("Génère un poème sur le thème du courage", do_sample = True, max_length = 200, min_length = 150, temperature = 1, top_k = 80)

[{'generated_text': "Génère un poème sur le thème du courage, tenus d'appel, du fait, de sa jeunesse... À notre avis, ce serait une minute au moment où nous pendons à ne pas me faireJean Alondeau? [R] [L] [J] \n\n\nRendu publiquement \nle\nà Me CHAMBREUJUE\nTRIBUNAL JUDICIAIRE (RCEPIQUE)\n\nPar Mme [G] [S], [C] [H], Me Antoine D’OUBERT, Me Camille PLANET, Me Cédric DEGIMENT, [F] [M] [M] [R], Me Emmanuel BOTELION, Me Christophe Daouia, Me François DELANAC, \n\nAu nom du Peuple Français, \nAU NOM DU PEUPLE FRANÇAIS\n\n\n\nTR"}]

In [39]:
df.head()

,prompt,text
0,Compose une œuvre poétique sur Les deux bonnes...,Sonnet.<|newline|>La Débauche et la Mort sont ...
1,Crée un poème autour de : Je ne vois pas pourq...,Je ne vois pas pourquoi je ferais autre chose<...
2,Écris un poème sur le thème : J'ai tout donné ...,"On peut m'aimer sans honte,<|newline|>La couro..."
3,Donne-moi un poème avec pour thème principal :...,La dure épreuve va finir
4,Propose un poème sur le sujet suivant : À deux...,Vous avez un regard singulier et charmant ;<|n...


In [40]:
df.shape

(1330, 2)

In [41]:
df.text[0]

"Sonnet.<|newline|>La Débauche et la Mort sont deux aimables filles,<|newline|>Prodigues de baisers et riches de santé,<|newline|>Dont le flanc toujours vierge et drapé de guenilles<|newline|>Sous l'éternel labeur n'a jamais enfanté.<|newline|>Au poète sinistre, ennemi des familles,<|newline|>Favori de l'enfer, courtisan mal tenté,<|newline|>Tombeaux et lupanars montrent sous leurs charmilles<|newline|>Un lit que le remords n'a jamais fréquenté.<|newline|>Et la bière et l'alcôve en blasphèmes fécondes<|newline|>Nous offrent tour à tour, comme deux bonnes soeurs,<|newline|>De terribles plaisirs et d'affreuses douceurs.<|newline|>Quand veux-tu m'enterrer, Débauche aux bras immondes ?<|newline|>Ô Mort, quand viendras-tu, sa rivale en attraits,<|newline|>Sur ses myrtes infects enter tes noirs cyprès ?"

## ***Importer le tokenizer*** ##

In [42]:
from transformers import AutoTokenizer

# Charger le tokenizer
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer = AutoTokenizer.from_pretrained("Gragroo/gpt2-french-small-finetuned-2024-09-21")

# Afficher les token par défaut
# Afficher les tokens spéciaux par défaut
print("Tokens spéciaux par défaut :", tokenizer.special_tokens_map)

Tokens spéciaux par défaut : {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}


## ***Charger le modèle et séparer le token unk_token (par défaut <|endoftext|>) et le redéfinir comme <|unknown|>***

In [43]:
from transformers import AutoModelForCausalLM

# Ajouter un nouveau token spécial
tokenizer.add_special_tokens({"unk_token": "<|unknown|>"})

# Charger le modèle
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
model = AutoModelForCausalLM.from_pretrained("Gragroo/gpt2-french-small-finetuned-2024-09-21")

# Redimensionner les embeddings du modèle pour inclure le nouveau token
model.resize_token_embeddings(len(tokenizer))

# Vérifier les tokens spéciaux après modification
print("Tokens spéciaux :", tokenizer.special_tokens_map)

# Vérifier si le token '<|unknown|>' a été ajouté au vocabulaire
print("ID du token '<|unknown|>' :", tokenizer.convert_tokens_to_ids("<|unknown|>"))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Tokens spéciaux : {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|unknown|>', 'pad_token': '<|endoftext|>'}
ID du token '<|unknown|>' : 50257


In [44]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [45]:
print("Taille du vocabulaire du tokenizer :", len(tokenizer))
print("Taille du vocabulaire du modèle :", model.config.vocab_size)

Taille du vocabulaire du tokenizer : 50258
Taille du vocabulaire du modèle : 50258


## ***Ajouter les tokens préalablement créés au tokenizer***

In [46]:
# Ajouter les nouveaux tokens
new_tokens = {"additional_special_tokens": ["<|newline|>", "<|pad|>"]}

# Vérifier et ajouter le token de padding s'il n'existe pas
if tokenizer.pad_token is None:
    new_tokens["pad_token"] = "<|pad|>"

# Ajouter les tokens au tokenizer
tokenizer.add_special_tokens(new_tokens)

# Vérifier que les tokens ont été ajoutés
print("Tokens spéciaux :", tokenizer.special_tokens_map)


Tokens spéciaux : {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|unknown|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|newline|>', '<|pad|>']}


## ***Redimensionner le modèle*** ##

Après avoir ajouté les nouveaux tokens au tokenizer, il est essentiel de redimensionner les embeddings du modèle pour inclure les nouveaux tokens

In [47]:
# Redimensionner les embeddings du modèle
model.resize_token_embeddings(len(tokenizer))

# Vérifier la taille du vocabulaire après modification
print("Taille du vocabulaire du tokenizer :", len(tokenizer))
print("Taille du vocabulaire du modèle :", model.config.vocab_size)

Taille du vocabulaire du tokenizer : 50260
Taille du vocabulaire du modèle : 50260


In [48]:
df.head()

,prompt,text
0,Compose une œuvre poétique sur Les deux bonnes...,Sonnet.<|newline|>La Débauche et la Mort sont ...
1,Crée un poème autour de : Je ne vois pas pourq...,Je ne vois pas pourquoi je ferais autre chose<...
2,Écris un poème sur le thème : J'ai tout donné ...,"On peut m'aimer sans honte,<|newline|>La couro..."
3,Donne-moi un poème avec pour thème principal :...,La dure épreuve va finir
4,Propose un poème sur le sujet suivant : À deux...,Vous avez un regard singulier et charmant ;<|n...


In [49]:
print(df.columns)

Index(['prompt', 'text'], dtype='object')


In [50]:
df["prompt"]

,prompt
0,Compose une œuvre poétique sur Les deux bonnes...
1,Crée un poème autour de : Je ne vois pas pourq...
2,Écris un poème sur le thème : J'ai tout donné ...
3,Donne-moi un poème avec pour thème principal :...
4,Propose un poème sur le sujet suivant : À deux...
...,...
1404,Donne-moi un poème avec pour thème principal :...
1405,"Un poème inspiré de La mort d'une libellule, s..."
1406,"Un poème inspiré de À Mademoiselle ***, s'il t..."
1407,Un poème autour de J'ai rêvé de toi cette nuit...


GPT-Neo n'accepte qu'une seule séquence d'entrée à la fois. Par conséquent, il est nécessaire de fusionner les deux colonnes de texte en une seule séquence avant de les fournir au modèle.

In [51]:
df['combined'] = df['prompt'] + ' ' + df['text']

## ***Train Test Split*** ##

In [52]:
from sklearn.model_selection import train_test_split

# Supposons que 'df' soit votre DataFrame contenant les données
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

## ***Convertir le dataframe en hugging face dataframe*** ##

In [53]:
# Enlever l'index pour éviter que la création de la colonne '__index_level_0__'
df_reset = df.reset_index(drop=True)

# Convertir le dataset en datframe hugging face
from datasets import Dataset


train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
test_dataset = Dataset.from_pandas(test_df, preserve_index=False)

In [54]:
train_dataset[0]

{'prompt': "Invente un poème portant sur : Chant d'amour (II)",
 'text': "Telle, pour sommeiller, la blanche tourterelle<|newline|>Courbe son cou d'albâtre et ramène son aile<|newline|>Sur son oeil endormi !<|newline|>Le doux gémissement de son sein qui respire<|newline|>Se mêle au bruit plaintif de l'onde qui soupire<|newline|>À flots harmonieux ;<|newline|>Et l'ombre de ses cils, que le zéphyr soulève,<|newline|>Flotte légèrement comme l'ombre d'un rêve<|newline|>Qui passe sur ses yeux !<|newline|>.................................................<|newline|>Que ton sommeil est doux, ô vierge ! ô ma colombe !<|newline|>Comme d'un cours égal ton sein monte et retombe<|newline|>Avec un long soupir !<|newline|>Deux vagues que blanchit le rayon de la lune,<|newline|>D'un mouvement moins doux viennent l'une après l'une<|newline|>Murmurer et mourir !<|newline|>Laisse-moi respirer sur ces lèvres vermeilles<|newline|>Ce souffle parfumé !...Qu'ai-je fait ? Tu t'éveilles ",
 'combined': "Invente

In [55]:
train_dataset

Dataset({
    features: ['prompt', 'text', 'combined'],
    num_rows: 1064
})

In [56]:
test_dataset

Dataset({
    features: ['prompt', 'text', 'combined'],
    num_rows: 266
})

## ***Appliquer le tokenizer sur la colonne combined*** ##

In [57]:
# Définir une fonction pour tokenizer les données textuelles
# Cette fonction prend chaque exemple de texte brut et le convertit en tokens avec des réglages spécifiques.
def tokenize_function(dataset):
    # Concaténer les colonnes `prompt` et `text`

    return tokenizer(
        dataset['combined'],            # Texte brut à tokenizer (la clé "text" est extraite du dataset).
        truncation=True,             # Coupe les textes plus longs que la longueur maximale spécifiée (max_length).
        padding="max_length",        # Ajoute du padding pour que toutes les séquences aient la même longueur (max_length).
        max_length=1489              # Définit la longueur maximale des séquences (ajustée selon la taille du contexte).
    )

In [58]:
# Appliquer la fonction de tokenization à tout le dataset
# La méthode `map` applique `tokenize_function` à chaque exemple du dataset, avec une option batched
# (traitement de plusieurs exemples à la fois pour une meilleure efficacité).
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1064 [00:00<?, ? examples/s]

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

In [59]:
for example in tokenized_test_dataset["input_ids"]:
    if max(example) >= len(tokenizer):
        print(f"Token ID invalide détecté : {max(example)}")

In [60]:
for example in tokenized_test_dataset["input_ids"]:
    if len(example) > 1489:  # Votre max_length
        print(f"Séquence trop longue détectée : {len(example)}")


## ***Définir les arguments du modèle et entraîner le modèle*** ##

ATTENTION !!!!!!!!!!! Avant d'efffectuer l'entraînement taper dans le termminal : "accelerate config" (série de question pour configurer GPU, TPU ou CPU)

selectionner vaec les flèches et taper sur entrer (*)

emplacement fichier config : accelerate configuration saved at C:\Users\33760/.cache\huggingface\accelerate\default_config.yaml

In [61]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())


False
0


In [62]:
from accelerate import Accelerator

accelerator = Accelerator()
print(accelerator.state)

Distributed environment: DistributedType.XLA  Backend: xla
Num processes: 8
Process index: 0
Local process index: 0
Device: xla:0

Mixed precision type: no



In [63]:
def add_labels(batch):
    batch["labels"] = batch["input_ids"].copy()
    return batch

tokenized_train_dataset = tokenized_train_dataset.map(add_labels)
tokenized_test_dataset = tokenized_test_dataset.map(add_labels)


Map:   0%|          | 0/1064 [00:00<?, ? examples/s]

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
# Importer les modules nécessaires de la bibliothèque transformers
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

# Configurer les arguments d'entraînement pour le fine-tuning
training_args = TrainingArguments(
    output_dir="./gpt-neo-125M-fine-tuned-poetry",             # Répertoire pour sauvegarder les résultats et checkpoints
    eval_strategy="epoch",       # Évaluer le modèle à la fin de chaque époque
    learning_rate=5e-5,                # Taux d'apprentissage (learning rate)
    per_device_train_batch_size=4,     # Taille du batch par appareil (GPU ou CPU)
    num_train_epochs=1,                # Nombre total d'époques d'entraînement
    save_steps=500,                    # Sauvegarder le modèle toutes les 500 étapes
    save_total_limit=2,                # Conserver uniquement les 2 dernières sauvegardes
    fp16=False,                        # Activer l'entraînement en virgule flottante 16 bits (plus rapide si GPU compatible)
    logging_dir = "./logs",
)

# Configurer le Trainer, une classe de Hugging Face pour gérer l'entraînement
trainer = Trainer(
    model=model,                       # Modèle à fine-tuner
    args=training_args,                # Arguments d'entraînement configurés précédemment
    train_dataset=tokenized_train_dataset,    # Jeu de données tokenisé utilisé pour l'entraînement
    eval_dataset=tokenized_test_dataset,
)

# Lancer le processus d'entraînement
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
# Sauvegarder le modèle fine-tuné dans le répertoire : ./gpt-neo-125M-fine-tuned-poetry
trainer.save_model()

In [ ]:
tokenizer.save_pretrained("./gpt-neo-125M-fine-tuned-poetry/tokenizer")

In [ ]:
def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores['rouge1'].fmeasure


In [ ]:
def generate_poem(model, tokenizer, keyword, max_length=100):
    if not keyword.strip():
        raise ValueError("Le mot-clé ne peut pas être vide. Veuillez entrer un mot valide.")

    input_ids = tokenizer.encode(keyword, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_poem


In [ ]:
def calculate_bleu(reference, candidate):
    return sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)


In [ ]:
from transformers import GPT2LMHeadModel, GPTNeoForCausalLM, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
#from rouge_score import rouge_scorer

In [ ]:
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

gptneo_model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
gptneo_tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

tokenizer = AutoTokenizer.from_pretrained("./gpt-neo-125M-fine-tuned-poetry/tokenizer")
model = AutoModelForCausalLM.from_pretrained("./gpt-neo-125M-fine-tuned-poetry")



In [ ]:
keyword = input("Entrez un mot-clé pour générer un poème : ").strip()

print(keyword)

if len(keyword) == 0:
    print("Erreur : Le mot-clé ne peut pas être vide.")
    exit()

if len(keyword) > 50:
    print("Erreur : Le mot-clé est trop long. Veuillez entrer un mot ou une courte phrase.")
    exit()

# Générer des poèmes pour chaque modèle
gpt2_poem = generate_poem(gpt2_model, gpt2_tokenizer, keyword)
#gptneo_poem = generate_poem(gptneo_model, gptneo_tokenizer, keyword)
model_poem = generate_poem(model, gptneo_tokenizer, keyword)

# Calculer les scores BLEU et ROUGE
bleu_score = calculate_bleu(gpt2_poem, model_poem)
rouge_score = calculate_rouge(gpt2_poem, model_poem)

# Afficher les poèmes générés et les scores
print("\nPoème généré par GPT-2 :\n", gpt2_poem)
#print("\nPoème généré par GPT-Neo :\n", gptneo_poem)
print("\nPoème généré par model :\n", model_poem)
print("\nScore BLEU entre les poèmes générés : ", bleu_score)
print("Score ROUGE entre les poèmes générés : ", rouge_score)
